In [ ]:
imp = open('/home/lindb/pythonimports.py').read()
exec(imp)
from __future__ import division

In [ ]:
1+1

In [ ]:
# reqs
fqdir = '/home/lindb/scratch/seqcap_pine'
shdir = op.join(fqdir,'shfiles')
lensh = 500                               # number of sh files that can be qsubbed
cd(fqdir)

In [ ]:
import sys
"/".join([x for x in sys.argv[0].split("/")[:4]])

In [ ]:
gzfiles = !ls *.gz
gzfiles = [f for f in gzfiles if '.Pi_' in f and 'R1' in f]
len(gzfiles),sorted(gzfiles)

In [ ]:
# match seq pairs, alert if pair not found
seq_pairs = []
for f in gzfiles:
    assert 'R1' in f
    read2 = f.replace("_R1","_R2")
    if op.exists(read2):
        seq_pairs.append((op.abspath(f),op.abspath(read2)))
    else:
        !echo 'no pair for '$f >> $fqdir'/messages/msgs.txt'
len(seq_pairs)

In [ ]:
seq_pairs = [('/home/lindb/scratch/seqcap_pine/paired_HI.0941.007.BioO_4.Pi_53A3_R1.fastq.gz',
             '/home/lindb/scratch/seqcap_pine/paired_HI.0941.007.BioO_4.Pi_53A3_R2.fastq.gz'
            )]

In [ ]:
# make some dirs
shtrimDIR    = op.join(shdir,'trimmed_shfiles')
trimDIR      = op.join(fqdir,'trimmed')         # outfiles
for d in [shtrimDIR,trimDIR]:
    if not op.exists(d):
        os.makedirs(d)

# determine how many commands per sh file
if len(seq_pairs) <= lensh:
    # one command per sh file
    ceil = 1
else:
    # multiple commands per sh file
    ceil = math.ceil(len(seq_pairs)/lensh)

shcount = 0
fcount  = 0
tcount  = 0
text    = ''''''
for s in seq_pairs:
    r1    = op.abspath(s[0])
    r1out = op.join(trimDIR,op.basename(r1).replace('.fastq.gz','_trimmedandfiltered.fastq.gz'))
    r2    = op.abspath(s[1])
    r2out = op.join(trimDIR,op.basename(r2).replace('.fastq.gz','_trimmedandfiltered.fastq.gz'))
    cmd   = '''fastp -i %s -o %s \
-I %s -O %s \
-g \
--cut_window_size 5 \
--cut_mean_quality 30 \
--qualified_quality_phred 30 \
--unqualified_percent_limit 20 \
--n_base_limit 5 \
--length_required 75 \
-h "%s.html" \
--cut_by_quality3 \
--thread 1 \
--json "%s.json"

''' % (r1,
       r1out,
       r2,
       r2out,
       r1out.replace("_R1_trimmed.fastq.gz","_stats.html"),
       r1out.replace("_R1_trimmed.fastq.gz",".json")
      )
    text = text + cmd
    
    fcount += 1
    tcount += 1
    if fcount == ceil or tcount == len(seq_pairs):
        shz = str(shcount).zfill(3)
        header = '''#!/bin/bash
#PBS -N trim%s
#PBS -V
#PBS -S /bin/bash
#PBS -l nodes=1:ppn=1
#PBS -l procs=1
#PBS -l walltime=0:10:00
#PBS -l mem=600mb
''' % (shz)
        text = header + text
        filE = op.join(shtrimDIR,'trim_%s.sh' % str(shcount).zfill(3))
        with open(filE,'w') as o:
            o.write("%s" % text)
        shcount += 1
        fcount = 0
        text = ''''''

In [ ]:
shs = fs(shtrimDIR)
for f in shs:
    !qsub $f